# Python для автоматизации и анализа данных

## Занятие 13 — 09/12/2021

## Часть 1 — web-scraping (продолжение)

*Автор: Анастасия Паршина, НИУ ВШЭ* 

E-mail: a.a.parshina@ya.ru

In [2]:
import re
import requests
from bs4 import BeautifulSoup 
from time import sleep
import pandas as pd

### Задание 1

Дана ссылка на первую страницу [каталога книжных новинок](https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=). Соберите информацию о книгах с первых трех страниц. 

In [3]:
urls = []

for i in range(1,4):
    urls.append(f'https://www.bgshop.ru/catalog/group?id=444&page={i}&sort=1&instock=')
urls

['https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=',
 'https://www.bgshop.ru/catalog/group?id=444&page=2&sort=1&instock=',
 'https://www.bgshop.ru/catalog/group?id=444&page=3&sort=1&instock=']

In [4]:
page = requests.get(urls[0])
page.status_code

200

In [5]:
soup = BeautifulSoup(page.text)
#soup

In [6]:
full_urls = ['https://www.bgshop.ru' + r.get('href') for r in soup.find_all('a', {'id' : re.compile('p_title')})]
full_urls

['https://www.bgshop.ru/Catalog/GetFullDescription?id=10821500&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10818788&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10817983&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10817980&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10817872&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10817765&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10817241&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10815460&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10815450&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10815271&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10815228&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10813549&type=1']

In [7]:
# ссылка
url0 = full_urls[5]
url0

'https://www.bgshop.ru/Catalog/GetFullDescription?id=10817765&type=1'

In [8]:
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)
#soup0

In [9]:
main_info = soup0.find_all('div', {'id':'productMain'})[0]
main_info

<div class="row box" id="productMain">
<div class="col-md-4">
<a data-fancybox="gallery" href="/imagehandler.ashx?filename=10817765.jpg&amp;width=400"><img alt="Книга «Маньяк Гуревич» Рубина Д." class="product_photo zoomIn" src="/imagehandler.ashx?filename=10817765.jpg&amp;width=200" title="Маньяк Гуревич" width="200"/></a>
</div>
<div class="col-md-7">
<div class="item-desc">
<p class="goToDescription"><a class="scroll-to" href="/Catalog/Search?query=Рубина Д.&amp;page=1">
Дина Рубина             </a></p>
<h1 id="p_title_10817765">Маньяк Гуревич</h1>
<p class="item-status">в наличии</p>
<div id="price"><b>Специальная цена</b> в интернет-магазине: <span id="p_price_10817765" style="color: #a42921; font-size:1.2rem;">789,00 руб.</span></div>
<p class="buttons">
<span class="btn btn-primary" id="add_10817765" onclick="AddToBasket(10817765)"><i class="fa fa-shopping-cart"></i> В корзину</span>
<a class="btn btn-primary add_but" href="/Basket/Detail" id="added_10817765">Перейти в корзину <

In [10]:
author = main_info.find_all('p')[0].text.strip()
author

'Дина Рубина'

In [11]:
title = main_info.find_all('h1')[0].text.strip()
title

'Маньяк Гуревич'

In [12]:
status = main_info.find_all('p', {'class':'item-status'})[0].text.strip()
status

'в наличии'

In [13]:
price = main_info.find_all('div', {'id':'price'})[0].text.strip()
price

'Специальная цена в интернет-магазине: 789,00\xa0руб.'

In [14]:
price_clean = float(re.findall(r'\d+,\d+', price)[0].replace(',', '.'))
price_clean

789.0

А теперь для всех (трех) страниц с книгами:

In [15]:
books = {}

for u in urls:
  page = requests.get(u)
  soup = BeautifulSoup(page.text)
  sleep(1)

  full_urls = ['https://www.bgshop.ru' + r.get('href') for r in soup.find_all('a', {'id' : re.compile('p_title')})]

  for url0 in full_urls:
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    sleep(1)

    main_info = soup0.find_all('div', {'id':'productMain'})[0]
    author = main_info.find_all('p')[0].text.strip()

    title = main_info.find_all('h1')[0].text.strip()
    status = main_info.find_all('p', {'class':'item-status'})[0].text
    price = float(re.findall(r'\d+,\d+', main_info.find_all('div', {'id':'price'})[0].text)[0].replace(',', '.'))

    books[title] = [author, status, price, url0]

In [16]:
df = pd.DataFrame(books, index = ['Автор', 'Статус', 'Цена', 'Ссылка']).transpose()
df

,Автор,Статус,Цена,Ссылка
Жизнь в эпизодах,Катерина Анатольевна Шпица,,579,https://www.bgshop.ru/Catalog/GetFullDescripti...
"Три жизни Ангелы Меркель: обычная, пропедевтическая, триумфальная. Политическая и личная биография первой женщины — федерального канцлера",Александр Алексеевич Деревянченко,в наличии,647,https://www.bgshop.ru/Catalog/GetFullDescripti...
Парижские мальчики в сталинской Москве : документальный роман,Сергей Станиславович Беляков,в наличии,927,https://www.bgshop.ru/Catalog/GetFullDescripti...
На плотной земле. Стихотворения,Петр Николаевич Мамонов,в наличии,439,https://www.bgshop.ru/Catalog/GetFullDescripti...
Три дня Индиго,Сергей Васильевич Лукьяненко,в наличии,551,https://www.bgshop.ru/Catalog/GetFullDescripti...
Маньяк Гуревич,Дина Рубина,в наличии,789,https://www.bgshop.ru/Catalog/GetFullDescripti...
Священные места Индии,Борис Борисович Гребенщиков,в наличии,719,https://www.bgshop.ru/Catalog/GetFullDescripti...
Мой адмирал,Татьяна Хабенская,,659,https://www.bgshop.ru/Catalog/GetFullDescripti...
Индия. Полная история страны,Аманда Шампри,в наличии,463,https://www.bgshop.ru/Catalog/GetFullDescripti...
Намедни. Наша эра. 2011-2015,Леонид Геннадьевич Парфенов,в наличии,2855,https://www.bgshop.ru/Catalog/GetFullDescripti...


### Задание 2

Дана ссылка на [сайт газеты "Московский комсомолец"](https://www.mk.ru/news/). Соберите ссылки на новости первой страницы, заголовок. Пройдите по ссылкам и соберите количество просмотров, хэштеги статьи и время публикации. 

In [17]:
url = 'https://www.mk.ru/news/'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [18]:
url0 = soup.find_all('li', {'class':'news-listing__item'})[0].find('a').get('href')
url0

'https://www.mk.ru/incident/2021/12/11/v-peterburgskoy-stomatologii-umer-shestiletniy-rebenok.html'

In [19]:
title = soup.find_all('li', {'class':'news-listing__item'})[0].find('h3').text
title

'В петербургской стоматологии умер шестилетний ребенок'

In [20]:
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)

In [21]:
soup0.find_all('span', {'class':'meta__item_views'})[0].text.strip()

'749'

[Пример статьи без тегов](https://www.mk.ru/social/2021/12/09/passazhiry-bez-qrkodov-mogut-lishitsya-kompensacii-za-kuplennye-bilety.html)

In [22]:
soup0.find('div', {'class':'article__tag'}).text

'\nСанкт-Петербург\n'

In [23]:
re.findall(r'[^\n]+', soup0.find('div', {'class':'article__tag'}).text)

['Санкт-Петербург']

In [25]:
time = re.findall(r'\d{2}:\d{2}', soup0.find_all('time')[0].text)[0]
time

'23:39'

In [24]:
news = {}
for i in soup.find_all('li', {'class':'news-listing__item'})[:5]:
    url0 = i.find('a').get('href')
    title = i.find('h3').text
    
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    sleep(2)
    
    view = int(soup0.find_all('span', {'class':'meta__item_views'})[0].text.strip())
    time = re.findall(r'\d{2}:\d{2}', soup0.find_all('time')[0].text)[0]
    if soup0.find('div', {'class':'article__tag'}) is None:
        tags = ''
    else:
        tags = ', '.join(re.findall(r'[^\n]+', soup0.find('div', {'class':'article__tag'}).text))
        
    news[title] = [url0, time, view, tags]

In [26]:
# можно также сразу указывать, что у нас данные из словаря, orient='index' вместо transpose()
df = pd.DataFrame.from_dict(news, orient='index', columns = ['Ссылка','Время','Просмотры','Теги'])
df

,Ссылка,Время,Просмотры,Теги
В петербургской стоматологии умер шестилетний ребенок,https://www.mk.ru/incident/2021/12/11/v-peterb...,00:15,787,Санкт-Петербург
Конькобежка Фаткулина взяла золото на этапе Кубка мира,https://www.mk.ru/social/2021/12/11/konkobezhk...,00:03,71,"Сочи, Канада"
"В Пентагоне заявили, что Россия сохраняет военное присутствие у границ Украины",https://www.mk.ru/politics/2021/12/10/v-pentag...,23:56,611,"Россия, Украина, США"
Певица Наталья Орейро впервые воспользовалась российским паспортом,https://www.mk.ru/politics/2021/12/10/pevica-n...,23:44,768,"ВКонтакте, Россия, Аргентина"
Зеленский подписал закон о госбюджете Украины на 2022 год,https://www.mk.ru/economics/2021/12/10/zelensk...,23:39,660,"Пенсионный фонд, Владимир Зеленский, Украина"


## Часть 2 — Работа с API YouTube

API (Application Programming Interface или интерфейс программирования приложений) — это совокупность инструментов и функций в виде интерфейса для создания новых приложений, благодаря которому одна программа будет взаимодействовать с другой.  

Подробнее [тут](https://habr.com/ru/post/464261/).

На основе лекций курса Ротмистрова А.В. и Жучковой С.В. [Методология и методы исследований в социологии: современные методы анализа социологических данных](https://www.hse.ru/edu/courses/375301084).


[Документация к API YouTube](https://developers.google.com/youtube/v3/docs?hl=ru)

In [27]:
import googleapiclient.discovery as api

In [28]:
youtube = api.build('youtube', 'v3', developerKey = 'ВАШ КЛЮЧ')


In [29]:
request = youtube.search().list(
        part="snippet",
        maxResults=50,
        q="вакцинация Pfizer",
        type='video')
response = request.execute()

In [30]:
response

{'etag': '-4bsMXmtiqyF0deZI6d-eJv8FsM',
 'items': [{'etag': 'wVWzdfVfuv1YqcWTig2TQXyQh3U',
   'id': {'kind': 'youtube#video', 'videoId': '9vXJYvmrc9I'},
   'kind': 'youtube#searchResult',
   'snippet': {'channelId': 'UCWAK-dRtTEjnQnD96L6UMsQ',
    'channelTitle': 'РБК',
    'description': 'Подпишитесь на канал РБК: https://www.youtube.com/user/tvrbcnews?sub_confirmation=1 --------------------- Американская Pfizer объявила, что ее вакцина от ...',
    'liveBroadcastContent': 'none',
    'publishTime': '2020-11-23T05:37:21Z',
    'publishedAt': '2020-11-23T05:37:21Z',
    'thumbnails': {'default': {'height': 90,
      'url': 'https://i.ytimg.com/vi/9vXJYvmrc9I/default.jpg',
      'width': 120},
     'high': {'height': 360,
      'url': 'https://i.ytimg.com/vi/9vXJYvmrc9I/hqdefault.jpg',
      'width': 480},
     'medium': {'height': 180,
      'url': 'https://i.ytimg.com/vi/9vXJYvmrc9I/mqdefault.jpg',
      'width': 320}},
    'title': 'Вакцины от коронавируса Pfizer и «Спутник V» — что 

In [31]:
response['items'][0] # первое видео

{'etag': 'wVWzdfVfuv1YqcWTig2TQXyQh3U',
 'id': {'kind': 'youtube#video', 'videoId': '9vXJYvmrc9I'},
 'kind': 'youtube#searchResult',
 'snippet': {'channelId': 'UCWAK-dRtTEjnQnD96L6UMsQ',
  'channelTitle': 'РБК',
  'description': 'Подпишитесь на канал РБК: https://www.youtube.com/user/tvrbcnews?sub_confirmation=1 --------------------- Американская Pfizer объявила, что ее вакцина от ...',
  'liveBroadcastContent': 'none',
  'publishTime': '2020-11-23T05:37:21Z',
  'publishedAt': '2020-11-23T05:37:21Z',
  'thumbnails': {'default': {'height': 90,
    'url': 'https://i.ytimg.com/vi/9vXJYvmrc9I/default.jpg',
    'width': 120},
   'high': {'height': 360,
    'url': 'https://i.ytimg.com/vi/9vXJYvmrc9I/hqdefault.jpg',
    'width': 480},
   'medium': {'height': 180,
    'url': 'https://i.ytimg.com/vi/9vXJYvmrc9I/mqdefault.jpg',
    'width': 320}},
  'title': 'Вакцины от коронавируса Pfizer и «Спутник V» — что лучше и как они работают?'}}

In [32]:
response['items'][0]['snippet'] # и тут нас интересует title

{'channelId': 'UCWAK-dRtTEjnQnD96L6UMsQ',
 'channelTitle': 'РБК',
 'description': 'Подпишитесь на канал РБК: https://www.youtube.com/user/tvrbcnews?sub_confirmation=1 --------------------- Американская Pfizer объявила, что ее вакцина от ...',
 'liveBroadcastContent': 'none',
 'publishTime': '2020-11-23T05:37:21Z',
 'publishedAt': '2020-11-23T05:37:21Z',
 'thumbnails': {'default': {'height': 90,
   'url': 'https://i.ytimg.com/vi/9vXJYvmrc9I/default.jpg',
   'width': 120},
  'high': {'height': 360,
   'url': 'https://i.ytimg.com/vi/9vXJYvmrc9I/hqdefault.jpg',
   'width': 480},
  'medium': {'height': 180,
   'url': 'https://i.ytimg.com/vi/9vXJYvmrc9I/mqdefault.jpg',
   'width': 320}},
 'title': 'Вакцины от коронавируса Pfizer и «Спутник V» — что лучше и как они работают?'}

In [33]:
response['items'][0]['snippet']['title']

'Вакцины от коронавируса Pfizer и «Спутник V» — что лучше и как они работают?'

In [34]:
for k in response['items']:
    print(k['snippet']['title'])

Вакцины от коронавируса Pfizer и «Спутник V» — что лучше и как они работают?
В Великобритании от побочных эффектов вакцины &quot;Пфайзер&quot; скончалось больше людей, чем от &quot;АстраЗенеки
П&#39;ять речей, які потрібно знати про вакцину Pfizer
Всё о вакцине Pfizer | ДЕТАЛИ
Pfizer заявила о необходимости третьей дозы вакцины от COVID-19
BioNTech/Pfizer, AstraZeneca та інші - яка вакцина проти COVID-19 найкраща? | DW Ukrainian
Вакцина от Pfizer подходит не всем и имеет серьезные побочные эффекты - Россия 24
У женщин проявилась странная побочка от вакцины Pfizer. Факты - Россия 24
Создатели вакцины Pfizer/BioNTech: &quot;эта вакцина разрабатывалась 30 лет&quot;
Вакцинация Pfizer: где и как будут хранить препарат в столице
Как будет проходить вакцинация Pfizer?
Достоинства вакцины Pfizer перевешивают ее недостатки - эксперты
Мій досвід вакцинації Пфайзер від COVID 19 - побічні ефекти після вакцини Pfizer
В Европе разгорается громкий скандал вокруг регистрации вакцин Pfizer и Moderna.
«

In [35]:
df = pd.json_normalize(response['items'])
df.head()

,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
0,youtube#searchResult,wVWzdfVfuv1YqcWTig2TQXyQh3U,youtube#video,9vXJYvmrc9I,2020-11-23T05:37:21Z,UCWAK-dRtTEjnQnD96L6UMsQ,Вакцины от коронавируса Pfizer и «Спутник V» —...,Подпишитесь на канал РБК: https://www.youtube....,https://i.ytimg.com/vi/9vXJYvmrc9I/default.jpg,120,90,https://i.ytimg.com/vi/9vXJYvmrc9I/mqdefault.jpg,320,180,https://i.ytimg.com/vi/9vXJYvmrc9I/hqdefault.jpg,480,360,РБК,none,2020-11-23T05:37:21Z
1,youtube#searchResult,vOGiqUzWWHF7APJU3tu6t-pDv3I,youtube#video,85ZxGtPsggQ,2021-04-21T19:30:40Z,UCKonxxVHzDl55V7a9n_Nlgg,В Великобритании от побочных эффектов вакцины ...,"Свежий скандал мирового уровня. Вакцина ""Пфайз...",https://i.ytimg.com/vi/85ZxGtPsggQ/default.jpg,120,90,https://i.ytimg.com/vi/85ZxGtPsggQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/85ZxGtPsggQ/hqdefault.jpg,480,360,Новости на Первом Канале,none,2021-04-21T19:30:40Z
2,youtube#searchResult,KQRncbLknYqF4-HRu2_UqaWkEWY,youtube#video,H21QKzuX-bo,2020-11-11T16:20:12Z,UC-wWyFdk_txbZV8FKEk0V8A,"П&#39;ять речей, які потрібно знати про вакцин...",Американська фармацевтична компанія Pfizer зая...,https://i.ytimg.com/vi/H21QKzuX-bo/default.jpg,120,90,https://i.ytimg.com/vi/H21QKzuX-bo/mqdefault.jpg,320,180,https://i.ytimg.com/vi/H21QKzuX-bo/hqdefault.jpg,480,360,Радіо Свобода Україна,none,2020-11-11T16:20:12Z
3,youtube#searchResult,ObhXtNtxkesJFIg0Hgij-vxFmOs,youtube#video,ik71Y9OEzhQ,2020-11-20T10:00:10Z,UCBG57608Hukev3d0d-gvLhQ,Всё о вакцине Pfizer | ДЕТАЛИ,0:41 - Чем же так порадовала мировую обществен...,https://i.ytimg.com/vi/ik71Y9OEzhQ/default.jpg,120,90,https://i.ytimg.com/vi/ik71Y9OEzhQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/ik71Y9OEzhQ/hqdefault.jpg,480,360,Настоящее Время,none,2020-11-20T10:00:10Z
4,youtube#searchResult,svaMa7XsLcXsqKxAVHzbW8S8dpE,youtube#video,qlPE_J_VTxo,2021-07-12T17:26:02Z,UCnAFKvDuqBGkIfV8Vn0J_CQ,Pfizer заявила о необходимости третьей дозы ва...,В некоторых странах набирает обороты повторная...,https://i.ytimg.com/vi/qlPE_J_VTxo/default.jpg,120,90,https://i.ytimg.com/vi/qlPE_J_VTxo/mqdefault.jpg,320,180,https://i.ytimg.com/vi/qlPE_J_VTxo/hqdefault.jpg,480,360,Телеканал Хабар 24,none,2021-07-12T17:26:02Z


In [36]:
i = 0
while 'nextPageToken' in response.keys():
    
    request = youtube.search().list(
        part="snippet",
        maxResults=50,
        q="вакцинация Pfizer",
        type='video',
        pageToken=response['nextPageToken'])
    
    response = request.execute()
    
    i += 1
    print(f'Итерация {i}')
    
    df_add = pd.json_normalize(response['items'])
    df = pd.concat([df, df_add])
df

Итерация 1
Итерация 2
Итерация 3
Итерация 4
Итерация 5
Итерация 6
Итерация 7
Итерация 8
Итерация 9
Итерация 10
Итерация 11


,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
0,youtube#searchResult,wVWzdfVfuv1YqcWTig2TQXyQh3U,youtube#video,9vXJYvmrc9I,2020-11-23T05:37:21Z,UCWAK-dRtTEjnQnD96L6UMsQ,Вакцины от коронавируса Pfizer и «Спутник V» —...,Подпишитесь на канал РБК: https://www.youtube....,https://i.ytimg.com/vi/9vXJYvmrc9I/default.jpg,120,90,https://i.ytimg.com/vi/9vXJYvmrc9I/mqdefault.jpg,320,180,https://i.ytimg.com/vi/9vXJYvmrc9I/hqdefault.jpg,480,360,РБК,none,2020-11-23T05:37:21Z
1,youtube#searchResult,vOGiqUzWWHF7APJU3tu6t-pDv3I,youtube#video,85ZxGtPsggQ,2021-04-21T19:30:40Z,UCKonxxVHzDl55V7a9n_Nlgg,В Великобритании от побочных эффектов вакцины ...,"Свежий скандал мирового уровня. Вакцина ""Пфайз...",https://i.ytimg.com/vi/85ZxGtPsggQ/default.jpg,120,90,https://i.ytimg.com/vi/85ZxGtPsggQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/85ZxGtPsggQ/hqdefault.jpg,480,360,Новости на Первом Канале,none,2021-04-21T19:30:40Z
2,youtube#searchResult,KQRncbLknYqF4-HRu2_UqaWkEWY,youtube#video,H21QKzuX-bo,2020-11-11T16:20:12Z,UC-wWyFdk_txbZV8FKEk0V8A,"П&#39;ять речей, які потрібно знати про вакцин...",Американська фармацевтична компанія Pfizer зая...,https://i.ytimg.com/vi/H21QKzuX-bo/default.jpg,120,90,https://i.ytimg.com/vi/H21QKzuX-bo/mqdefault.jpg,320,180,https://i.ytimg.com/vi/H21QKzuX-bo/hqdefault.jpg,480,360,Радіо Свобода Україна,none,2020-11-11T16:20:12Z
3,youtube#searchResult,ObhXtNtxkesJFIg0Hgij-vxFmOs,youtube#video,ik71Y9OEzhQ,2020-11-20T10:00:10Z,UCBG57608Hukev3d0d-gvLhQ,Всё о вакцине Pfizer | ДЕТАЛИ,0:41 - Чем же так порадовала мировую обществен...,https://i.ytimg.com/vi/ik71Y9OEzhQ/default.jpg,120,90,https://i.ytimg.com/vi/ik71Y9OEzhQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/ik71Y9OEzhQ/hqdefault.jpg,480,360,Настоящее Время,none,2020-11-20T10:00:10Z
4,youtube#searchResult,svaMa7XsLcXsqKxAVHzbW8S8dpE,youtube#video,qlPE_J_VTxo,2021-07-12T17:26:02Z,UCnAFKvDuqBGkIfV8Vn0J_CQ,Pfizer заявила о необходимости третьей дозы ва...,В некоторых странах набирает обороты повторная...,https://i.ytimg.com/vi/qlPE_J_VTxo/default.jpg,120,90,https://i.ytimg.com/vi/qlPE_J_VTxo/mqdefault.jpg,320,180,https://i.ytimg.com/vi/qlPE_J_VTxo/hqdefault.jpg,480,360,Телеканал Хабар 24,none,2021-07-12T17:26:02Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,youtube#searchResult,tW8wBt6HNijRtAKnd1Cmyc2qlvM,youtube#video,bHAXo_a0GjU,2021-06-16T16:00:16Z,UCTJR_NKy4a_7d0evcjFiqPA,Почему обязательная вакцинация необходима (Ана...,"Интервью Путина NBC, ковидные ограничения, вст...",https://i.ytimg.com/vi/bHAXo_a0GjU/default.jpg,120,90,https://i.ytimg.com/vi/bHAXo_a0GjU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/bHAXo_a0GjU/hqdefault.jpg,480,360,Радио АВРОРА,none,2021-06-16T16:00:16Z
32,youtube#searchResult,hokZkE3C0mNbi-18AXA0EL9K-3A,youtube#video,8AV3Fh-MKJ4,2017-09-21T15:29:02Z,UCIFxeXrXXgQUm9BU3HvLTbw,Билл Гейтс подтверждает - Вакцинация служит дл...,Билл Гейтс в интервью доктору медицине Sanjay ...,https://i.ytimg.com/vi/8AV3Fh-MKJ4/default.jpg,120,90,https://i.ytimg.com/vi/8AV3Fh-MKJ4/mqdefault.jpg,320,180,https://i.ytimg.com/vi/8AV3Fh-MKJ4/hqdefault.jpg,480,360,Правда о прививках,none,2017-09-21T15:29:02Z
33,youtube#searchResult,MR7ROXjQPTczxHKOsSwgCRONUkM,youtube#video,XbmPIWshto0,2021-08-04T07:45:42Z,UCyTvW2rSEdREMGLy4um4AHA,Вакцинация как форма расизма.,Взгляд из глубинки. Помощь телеканалу: https:/...,https://i.ytimg.com/vi/XbmPIWshto0/default.jpg,120,90,https://i.ytimg.com/vi/XbmPIWshto0/mqdefault.jpg,320,180,https://i.ytimg.com/vi/XbmPIWshto0/hqdefault.jpg,480,360,Ивановское общественное телевидение,none,2021-08-04T07:45:42Z
34

In [37]:
df.index = range(1, len(df)+1)
df = df.drop_duplicates('id.videoId')
df.shape

(574, 20)

In [ ]:
# df.to_excel('Видео.xlsx')